### Load Data

In [1]:
import warnings
warnings.filterwarnings("ignore", message=".*'dropout_adj' is deprecated, use 'dropout_edge' instead.*")

In [2]:
import scipy.sparse as sp
import numpy as np
import json
import torch
import torch.nn.functional as F

In [3]:
# load data
adj = sp.load_npz('../CSE881_data_2024/adj.npz')
features  = np.load('../CSE881_data_2024/features.npy')
labels = np.load('../CSE881_data_2024/labels.npy')
splits = json.load(open('../CSE881_data_2024/splits.json'))
idx_train, idx_test = splits['idx_train'], splits['idx_test']

In [4]:
# transfer adjacency matrix into edge index
from torch_geometric.utils import from_scipy_sparse_matrix

edge_index = from_scipy_sparse_matrix(adj)
print("There are", edge_index[0].size(1), "edges in total in the graph\n")

print(torch.unique(edge_index[1]))
print("These edges are not weighted.")

There are 10100 edges in total in the graph

tensor([1.])
These edges are not weighted.


In [5]:
print("There are", len(features), "nodes in the graph.")
num_classes = len(np.unique(labels))
print("Each node can be one of", num_classes, "classes.")
print("Training set size:", len(idx_train))
print("Test set size:", len(idx_test))

There are 2480 nodes in the graph.
Each node can be one of 7 classes.
Training set size: 496
Test set size: 1984


In [6]:
device = 'cuda'

In [7]:
features = torch.from_numpy(features).float()
num_features = len(features[0])
print("Number of features:", num_features)

Number of features: 1390


In [8]:
features = features.to(device)
edge_index = edge_index[0].to(device)

### Supurvised Contrastive Learning Preparation

Method 'GRACE': Zhu et al., Deep Graph Contrastive Representation Learning, GRL+@ICML, 2020

https://arxiv.org/abs/2006.04131

Method 'SupCon': P. Khosla et al., Supervised Contrastive Learning, NeurIPS, 2020

https://arxiv.org/abs/2004.11362

In [9]:
import GCL.augmentors as A
import GCL.losses as L
from GCL.models import DualBranchContrast

In [10]:
class Encoder(torch.nn.Module):
    def __init__(self, encoder, augmentor):
        super(Encoder, self).__init__()
        self.encoder = encoder
        self.augmentor = augmentor

    def forward(self, x, edge_index, edge_weight=None):
        aug1, aug2 = self.augmentor
        x1, edge_index1, edge_weight1 = aug1(x, edge_index)
        x2, edge_index2, edge_weight2 = aug2(x, edge_index)
        z = self.encoder(x, edge_index)
        z1 = self.encoder(x1, edge_index1)
        z2 = self.encoder(x2, edge_index2)
        return z, z1, z2

In [11]:
def contrastive_mask(idx_train_sub, train_labels_sub):
    
    num_nodes = len(features)

    # create extra_pos_mask
    # initialize a 2480 x 2480 matrix of False
    extra_pos_mask = torch.zeros((num_nodes, num_nodes), dtype=torch.bool).to(device)

    # create a temporary full label tensor initialized with a dummy label and place the known labels
    full_labels = torch.full((num_nodes,), -1, dtype=train_labels_sub.dtype).to(device)
    full_labels[idx_train_sub] = train_labels_sub

    # iterate through each known label and update the label_matrix to True by finding nodes with the same label
    for i, label in zip(idx_train_sub, train_labels_sub):
        same_label_indices = torch.where(full_labels == label)[0]
        extra_pos_mask[i, same_label_indices] = True
        extra_pos_mask[same_label_indices, i] = True
    extra_pos_mask.fill_diagonal_(False)

    # pos_mask: [N, 2N] for both inter-view and intra-view samples
    extra_pos_mask = torch.cat([extra_pos_mask, extra_pos_mask], dim=1).to(device)
    # fill inter-view positives only; pos_mask for intra-view samples should have False in diagonal
    extra_pos_mask.fill_diagonal_(True)

    # create extra_neg_mask
    # initialize a 2480 x 2480 matrix of True
    extra_neg_mask = torch.ones((num_nodes, num_nodes), dtype=torch.bool).to(device)

    # iterate through each known label and update the label_matrix to False by finding nodes with the same label
    for i, label in zip(idx_train_sub, train_labels_sub):
        same_label_indices = torch.where(full_labels == label)[0]
        extra_neg_mask[i, same_label_indices] = False
        extra_neg_mask[same_label_indices, i] = False

    # set the diagonal to False since a sample cannot be a negative of itself
    extra_neg_mask.fill_diagonal_(False)

    # neg_mask: [N, 2N] for both inter-view and intra-view samples
    extra_neg_mask = torch.cat([extra_neg_mask, extra_neg_mask], dim=1).to(device)
    
    return extra_pos_mask, extra_neg_mask

In [12]:
# augumentation of the graph
aug1 = A.Compose([A.EdgeRemoving(pe=0.1), A.FeatureDropout(pf=0.1), A.NodeDropping(pn=0.1)])
aug2 = A.Compose([A.EdgeRemoving(pe=0.1), A.FeatureDropout(pf=0.1), A.NodeDropping(pn=0.1)])

In [13]:
# contrastive loss function
contrast_loss = DualBranchContrast(loss=L.InfoNCE(tau=0.2), mode='L2L', intraview_negs=True).to(device)

### Hyperparameter Tuner

In [14]:
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [15]:
class HyperparameterTuner:
    def __init__(self, model_structure, optimizer_lr, scheduler_factor, scheduler_patience, 
                 epoches, lambda_reg, output_interval=4, features=features, edge_index=edge_index,
                 labels=labels, idx=idx_train, method='SupCon'):
        """
        Used for hyperparameter tuning with spliting 20% of training set as validation set
        
        :param model_structure: The function to define model structure
        :param optimizer_lr: The initial learning rate for the optimizer
        :param scheduler_factor: The factor by which the learning rate will be reduced
        :param scheduler_patience: The number of epochs with no improvement after which learning rate will be reduced
        :param epoches: The number of epoches to train
        :param lambda_reg: Hyperparameter to control ratio of contrastive loss
        :param output_interval: The interval to ouput the training result
        :param features: The features of the dataset
        :param edge_index: The edge index tensor describing graph connectivity
        :param labels: The labels for data points in the dataset
        :param idx: Indices of the dataset to be used for splitting to train_set_sub and validation set
        :param method: Contrastive learning method: SupCon or GRACE

        """
        
        self.model_structure = model_structure
        self.model = self.model_structure() 
        
        self.optimizer_lr = optimizer_lr
        self.scheduler_factor = scheduler_factor
        self.scheduler_patience = scheduler_patience
        self.epoches = epoches
        self.lambda_reg = lambda_reg
        
        self.output_interval = output_interval
        self.features = features
        self.edge_index = edge_index
        self.labels = labels
        self.idx = idx
        self.method = method
     
        # Splitting the dataset into training and validation sets
        self.idx_train_sub, self.idx_val, train_labels_sub, val_labels = train_test_split(
            idx, labels, test_size=0.2, random_state=123, stratify=labels)
        
        # Converting labels to tensor
        self.train_labels_sub = torch.from_numpy(train_labels_sub).long().to(device)
        self.val_labels = torch.from_numpy(val_labels).long().to(device)
        
        self.encoder_model = Encoder(encoder=self.model, augmentor=(aug1, aug2)).to(device)
        self.optimizer = torch.optim.Adam(self.encoder_model.parameters(), optimizer_lr)
        self.scheduler = ReduceLROnPlateau(self.optimizer, mode='min', factor=scheduler_factor, patience=scheduler_patience)
        
        self.supervised_criterion = torch.nn.CrossEntropyLoss()
        self.contrast_criterion = contrast_loss
        self.extra_pos_mask, self.extra_neg_mask = contrastive_mask(self.idx_train_sub, self.train_labels_sub)
    
    
    def train(self):
        """
        Trains the model on the training dataset.
        """
        self.encoder_model.train()
        self.optimizer.zero_grad()
        z, z1, z2 = self.encoder_model(self.features, self.edge_index)
        
        if self.method == 'SupCon':
            contrast_loss = self.contrast_criterion(h1=z1, h2=z2, extra_pos_mask=self.extra_pos_mask, extra_neg_mask=self.extra_neg_mask)
        elif self.method == 'GRACE':
            contrast_loss = self.contrast_criterion(h1=z1, h2=z2)
        else:
            raise ValueError("Unsupported method specified. Please choose 'SupCon' or 'GRACE'.")
        supervised_loss = self.supervised_criterion(z[self.idx_train_sub], self.train_labels_sub)
        
        total_loss = supervised_loss + self.lambda_reg * contrast_loss
        total_loss.backward()
        self.optimizer.step()
        
        return contrast_loss, supervised_loss
    
    
    def test(self):
        """
        Evaluates the model on the validation subset.
        """
        self.encoder_model.eval()
        with torch.no_grad():
            out = self.encoder_model(self.features, self.edge_index)
            pred = out[0].argmax(dim=1)
            
            train_correct = pred[self.idx_train_sub] == self.train_labels_sub
            train_acc = int(train_correct.sum()) / len(self.idx_train_sub)
            
            val_correct = pred[self.idx_val] == self.val_labels
            val_acc = int(val_correct.sum()) / len(self.idx_val)
            
            val_loss = self.supervised_criterion(out[0][self.idx_val], self.val_labels).item()
            
        return train_acc, val_acc, val_loss

    
    def run(self):
        """
        Executes the training and validation process, adjusting the learning rate and lambda as needed.
        """   
        for epoch in range(self.epoches):
            contrast_loss, supervised_loss = self.train()
            train_acc, val_acc, val_loss = self.test()
            self.scheduler.step(val_loss)
            
            if epoch % self.output_interval == 0:
                print(f'Epoch: {epoch:03d}, Con Loss: {contrast_loss:.2f}, Sup Loss: {supervised_loss:.2f}, Train Acc: {train_acc:.2f}, Val Loss: {val_loss:.2f}, Val Acc: {val_acc:.2f}')

    """
    Example usage:
    tuner = HyperparameterTuner(model_structure, optimizer_lr=0.001, 
    scheduler_factor=0.1, scheduler_patience=10, epoches=100, lambda=0.9,
    output_interval=4, features=features, edge_index_tensor=edge_index_tensor, 
    labels=labels, idx=idx_train, method='SupCon')
    tuner.run()
    """

### Model Evaluator

In [16]:
from sklearn.model_selection import StratifiedKFold

In [17]:
class ModelEvaluator:
    def __init__(self, model_structure, optimizer_lr, scheduler_factor, scheduler_patience, 
                 epoches, lambda_reg, features=features, edge_index=edge_index, labels=labels, 
                 idx=idx_train, n_folds=5, device='cuda', method='SupCon'):
        """
        5-fold cross-validation to evaluate the model with multiple rounds
        
        :param model_structure: The function to define model structure
        :param optimizer_lr: The initial learning rate for the optimizer
        :param scheduler_factor: The factor by which the learning rate will be reduced
        :param scheduler_patience: The number of epochs with no improvement after which learning rate will be reduced
        :param epoches: The number of epoch to train
        :param lambda_reg: Hyperparameter to control ratio of contrastive loss
        :param features: The features of the dataset
        :param edge_index: The edge index tensor describing graph connectivity
        :param labels: The labels for data points in the dataset
        :param idx: Indices of the dataset to be used for splitting to train_set_sub and validation set
        :param cv_rounds: The round of cross-validation
        :param n_folds: Number of folds for cross-validation
        :param method: Contrastive learning method: SupCon or GRACE

        """
        self.model_structure = model_structure
        
        self.optimizer_lr = optimizer_lr
        self.scheduler_factor = scheduler_factor
        self.scheduler_patience = scheduler_patience
        self.epoches = epoches
        self.lambda_reg = lambda_reg
        
        self.features = features
        self.edge_index = edge_index
        self.labels = np.array(labels)
        self.idx = np.array(idx)
        
        self.n_folds = n_folds
        self.device = device
        
        self.encoder_models = []
        
        self.supervised_criterion = torch.nn.CrossEntropyLoss()
        self.contrast_criterion = contrast_loss
        self.method = method
        
        
    def train(self, encoder_model, optimizer, features, edge_index, 
              train_labels_sub, idx_train_sub, extra_pos_mask, extra_neg_mask):
        """
        Trains the model on the training dataset.
        """
        encoder_model.train()
        optimizer.zero_grad()
        z, z1, z2 = encoder_model(features, edge_index)
        
        if self.method == 'SupCon':
            contrast_loss = self.contrast_criterion(h1=z1, h2=z2, extra_pos_mask=extra_pos_mask, extra_neg_mask=extra_neg_mask)
        elif self.method == 'GRACE':
            contrast_loss = self.contrast_criterion(h1=z1, h2=z2)
        else:
            raise ValueError("Unsupported method specified. Please choose 'SupCon' or 'GRACE'.")
        supervised_loss = self.supervised_criterion(z[idx_train_sub], train_labels_sub)
        
        total_loss = supervised_loss + self.lambda_reg * contrast_loss
        total_loss.backward()
        optimizer.step()
        
        return contrast_loss, supervised_loss
    
    
    def test(self, encoder_model, features, edge_index, val_labels, idx_val):
        """
        Evaluates the model on the validation subset.
        """
        encoder_model.eval()
        with torch.no_grad():
            out = encoder_model(features, edge_index)
            pred = out[0].argmax(dim=1)
            
            val_correct = pred[idx_val] == val_labels
            val_acc = int(val_correct.sum()) / len(idx_val)
            
            val_loss = self.supervised_criterion(out[0][idx_val], val_labels).item()
            
        return val_acc, val_loss

    
    def evaluate(self):
        """
        Cross-validation to evaluate the model, print accuracy of each model, 
        average accuracy per round, and overall average accuracy. 
        Models and their accuracies are saved in a dictionary.
        """       
        cv_acc = []
        cv_loss = []
            
        kf = StratifiedKFold(n_splits=self.n_folds, shuffle=True, random_state=123)
        fold_count = 0
        for train_index_sub, val_index in kf.split(self.idx, self.labels):
            fold_count += 1
                
            # Create CV idx and labels for current round
            idx_train_sub, idx_val = self.idx[train_index_sub], self.idx[val_index]
            train_labels_sub, val_labels = self.labels[train_index_sub], self.labels[val_index]
                
            train_labels_sub = torch.from_numpy(train_labels_sub).long().to(self.device)
            val_labels = torch.from_numpy(val_labels).long().to(self.device)
                
            cur_extra_pos_mask, cur_extra_neg_mask = contrastive_mask(idx_train_sub, train_labels_sub)
                
            # Instantiate a new encoder model for each fold
            cur_model = self.model_structure().to(self.device) 
            cur_encoder_model = Encoder(encoder=cur_model, augmentor=(aug1, aug2)).to(device)
                
            # Setting up the optimizer, scheduler, and loss function
            cur_optimizer = torch.optim.Adam(cur_encoder_model.parameters(), lr=self.optimizer_lr)
            cur_scheduler = ReduceLROnPlateau(cur_optimizer, mode='min', factor=self.scheduler_factor, patience=self.scheduler_patience)
                
            for epoch in range(self.epoches):
                contrast_loss, supervised_loss = self.train(cur_encoder_model, cur_optimizer, 
                                        self.features, self.edge_index, train_labels_sub, 
                                        idx_train_sub, cur_extra_pos_mask, cur_extra_neg_mask)
                val_acc, val_loss = self.test(cur_encoder_model, self.features, 
                                                self.edge_index, val_labels, idx_val)
                cur_scheduler.step(val_loss)
                
            cv_acc.append(val_acc)
            cv_loss.append(val_loss)
            print(f"Fold {fold_count} - Val Accuracy: {val_acc:.3f}, Val Loss: {val_loss:.3f}")
                
        avg_val_acc = np.mean(cv_acc)
        avg_val_loss = np.mean(cv_loss)
        print(f"Average Val Accuracy: {avg_val_acc:.4f}, Average Val Loss: {avg_val_loss:.4f}")

    """
    Example usage:
    evaluator = HyperparameterTuner(model_structure, optimizer_lr=0.001, 
    scheduler_factor=0.1, scheduler_patience=10, epochs=100, lambda_reg=0.9,
    features=features, edge_index_tensor=edge_index_tensor, 
    labels=labels, idx=idx_train, n_folds=5, method='SupCon')
    models = evaluator.evaluate()
    """

### GCN

In [18]:
from torch_geometric.nn import GCN

In [19]:
def GCN_model():
    torch.manual_seed(123)
    return GCN(in_channels=num_features, hidden_channels=512, 
               out_channels=num_classes, num_layers=3, dropout=0.5, act='relu').to(device)

In [20]:
tuner = HyperparameterTuner(GCN_model, optimizer_lr=0.0005, epoches=210, scheduler_factor=0.1, 
                            scheduler_patience=1000, lambda_reg=2, output_interval=8)
tuner.run()

Epoch: 000, Con Loss: 9.22, Sup Loss: 1.94, Train Acc: 0.15, Val Loss: 1.94, Val Acc: 0.15
Epoch: 008, Con Loss: 8.51, Sup Loss: 1.93, Train Acc: 0.15, Val Loss: 1.94, Val Acc: 0.15
Epoch: 016, Con Loss: 8.51, Sup Loss: 1.87, Train Acc: 0.15, Val Loss: 1.88, Val Acc: 0.15
Epoch: 024, Con Loss: 8.50, Sup Loss: 1.78, Train Acc: 0.15, Val Loss: 1.79, Val Acc: 0.15
Epoch: 032, Con Loss: 8.49, Sup Loss: 1.67, Train Acc: 0.42, Val Loss: 1.69, Val Acc: 0.34
Epoch: 040, Con Loss: 8.45, Sup Loss: 1.54, Train Acc: 0.57, Val Loss: 1.56, Val Acc: 0.50
Epoch: 048, Con Loss: 8.34, Sup Loss: 1.36, Train Acc: 0.64, Val Loss: 1.39, Val Acc: 0.59
Epoch: 056, Con Loss: 8.13, Sup Loss: 1.17, Train Acc: 0.68, Val Loss: 1.21, Val Acc: 0.62
Epoch: 064, Con Loss: 8.03, Sup Loss: 0.99, Train Acc: 0.71, Val Loss: 1.03, Val Acc: 0.66
Epoch: 072, Con Loss: 7.88, Sup Loss: 0.85, Train Acc: 0.73, Val Loss: 0.91, Val Acc: 0.69
Epoch: 080, Con Loss: 7.82, Sup Loss: 0.76, Train Acc: 0.77, Val Loss: 0.83, Val Acc: 0.71

In [21]:
evaluator = ModelEvaluator(GCN_model, optimizer_lr=0.0005, epoches=210, scheduler_factor=0.1, 
                            scheduler_patience=1000, lambda_reg=2)
evaluator.evaluate()

Fold 1 - Val Accuracy: 0.830, Val Loss: 0.533
Fold 2 - Val Accuracy: 0.838, Val Loss: 0.546
Fold 3 - Val Accuracy: 0.869, Val Loss: 0.380
Fold 4 - Val Accuracy: 0.859, Val Loss: 0.473
Fold 5 - Val Accuracy: 0.848, Val Loss: 0.568
Average Val Accuracy: 0.8488, Average Val Loss: 0.5000


### GraphSAGE

In [22]:
from torch_geometric.nn import GraphSAGE

In [23]:
def GraphSAGE_model():
    torch.manual_seed(123)
    return GraphSAGE(in_channels=num_features, hidden_channels=256, 
               out_channels=num_classes, num_layers=3, dropout=0.5, act='relu').to(device)

In [24]:
tuner = HyperparameterTuner(GraphSAGE_model, optimizer_lr=0.0003, epoches=145, scheduler_factor=0.2, 
                            scheduler_patience=1000, output_interval=8, lambda_reg=0.8)
tuner.run()

Epoch: 000, Con Loss: 9.13, Sup Loss: 1.96, Train Acc: 0.05, Val Loss: 1.96, Val Acc: 0.05
Epoch: 008, Con Loss: 8.54, Sup Loss: 1.99, Train Acc: 0.11, Val Loss: 1.99, Val Acc: 0.10
Epoch: 016, Con Loss: 8.52, Sup Loss: 1.97, Train Acc: 0.11, Val Loss: 1.98, Val Acc: 0.09
Epoch: 024, Con Loss: 8.51, Sup Loss: 1.95, Train Acc: 0.11, Val Loss: 1.95, Val Acc: 0.09
Epoch: 032, Con Loss: 8.51, Sup Loss: 1.91, Train Acc: 0.19, Val Loss: 1.93, Val Acc: 0.11
Epoch: 040, Con Loss: 8.50, Sup Loss: 1.87, Train Acc: 0.28, Val Loss: 1.89, Val Acc: 0.21
Epoch: 048, Con Loss: 8.49, Sup Loss: 1.82, Train Acc: 0.54, Val Loss: 1.84, Val Acc: 0.33
Epoch: 056, Con Loss: 8.47, Sup Loss: 1.72, Train Acc: 0.73, Val Loss: 1.76, Val Acc: 0.56
Epoch: 064, Con Loss: 8.33, Sup Loss: 1.57, Train Acc: 0.84, Val Loss: 1.61, Val Acc: 0.75
Epoch: 072, Con Loss: 8.12, Sup Loss: 1.33, Train Acc: 0.88, Val Loss: 1.39, Val Acc: 0.81
Epoch: 080, Con Loss: 8.06, Sup Loss: 1.06, Train Acc: 0.89, Val Loss: 1.12, Val Acc: 0.81

In [25]:
evaluator = ModelEvaluator(GraphSAGE_model, optimizer_lr=0.0003, epoches=145, scheduler_factor=0.2, 
                            scheduler_patience=1000, lambda_reg=0.8)
evaluator.evaluate()

Fold 1 - Val Accuracy: 0.810, Val Loss: 0.506
Fold 2 - Val Accuracy: 0.808, Val Loss: 0.642
Fold 3 - Val Accuracy: 0.889, Val Loss: 0.368
Fold 4 - Val Accuracy: 0.848, Val Loss: 0.485
Fold 5 - Val Accuracy: 0.869, Val Loss: 0.572
Average Val Accuracy: 0.8448, Average Val Loss: 0.5148


### GAT

In [26]:
from torch_geometric.nn import GATConv

In [27]:
class GAT(torch.nn.Module):
    def __init__(self, in_channels, out_channels, hidden_channels, heads, dropout):
        super().__init__()
        self.gat1 = GATConv(in_channels, hidden_channels, heads=heads, dropout=dropout) 
        self.gat2 = GATConv(hidden_channels*heads, hidden_channels, heads=heads, dropout=dropout) 
        self.gat3 = GATConv(hidden_channels*heads, out_channels, concat=False,
                             heads=1, dropout=dropout)  
    
    def forward(self, x, edge_index):
        x = self.gat1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.7, training=self.training)
        x = self.gat2(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.7, training=self.training)
        x = self.gat3(x, edge_index)
        return x

In [28]:
def GAT_model():
    torch.manual_seed(123)
    return GAT(in_channels=num_features, hidden_channels=32, 
               out_channels=num_classes, heads=16, dropout=0.7).to(device)

In [29]:
tuner = HyperparameterTuner(GAT_model, optimizer_lr=0.0008, epoches=67, scheduler_factor=0.3, 
                            scheduler_patience=1000, output_interval=3, lambda_reg=0)
tuner.run()

Epoch: 000, Con Loss: 9.27, Sup Loss: 2.07, Train Acc: 0.34, Val Loss: 1.89, Val Acc: 0.33
Epoch: 003, Con Loss: 10.00, Sup Loss: 1.83, Train Acc: 0.41, Val Loss: 1.68, Val Acc: 0.38
Epoch: 006, Con Loss: 9.72, Sup Loss: 1.64, Train Acc: 0.59, Val Loss: 1.50, Val Acc: 0.51
Epoch: 009, Con Loss: 9.68, Sup Loss: 1.52, Train Acc: 0.76, Val Loss: 1.35, Val Acc: 0.68
Epoch: 012, Con Loss: 9.59, Sup Loss: 1.37, Train Acc: 0.81, Val Loss: 1.21, Val Acc: 0.78
Epoch: 015, Con Loss: 9.57, Sup Loss: 1.35, Train Acc: 0.82, Val Loss: 1.08, Val Acc: 0.79
Epoch: 018, Con Loss: 9.58, Sup Loss: 1.17, Train Acc: 0.84, Val Loss: 0.95, Val Acc: 0.78
Epoch: 021, Con Loss: 9.51, Sup Loss: 1.13, Train Acc: 0.86, Val Loss: 0.85, Val Acc: 0.80
Epoch: 024, Con Loss: 9.56, Sup Loss: 1.12, Train Acc: 0.88, Val Loss: 0.76, Val Acc: 0.83
Epoch: 027, Con Loss: 9.49, Sup Loss: 1.04, Train Acc: 0.89, Val Loss: 0.68, Val Acc: 0.85
Epoch: 030, Con Loss: 9.39, Sup Loss: 0.97, Train Acc: 0.91, Val Loss: 0.63, Val Acc: 0.8

In [30]:
evaluator = ModelEvaluator(GAT_model, optimizer_lr=0.0008, epoches=67, scheduler_factor=0.3, 
                            scheduler_patience=1000, lambda_reg=0)
evaluator.evaluate()

Fold 1 - Val Accuracy: 0.820, Val Loss: 0.513
Fold 2 - Val Accuracy: 0.818, Val Loss: 0.614
Fold 3 - Val Accuracy: 0.848, Val Loss: 0.395
Fold 4 - Val Accuracy: 0.869, Val Loss: 0.467
Fold 5 - Val Accuracy: 0.848, Val Loss: 0.527
Average Val Accuracy: 0.8408, Average Val Loss: 0.5033


### GAT - v2

In [31]:
from torch_geometric.nn import GATv2Conv

In [32]:
class GATv2(torch.nn.Module):
    def __init__(self, in_channels, out_channels, hidden_channels, heads, dropout):
        super().__init__()
        self.gatv21 = GATv2Conv(num_features, hidden_channels, heads=heads, dropout=dropout)
        self.gatv22 = GATv2Conv(hidden_channels*heads, hidden_channels, heads=heads, dropout=dropout)
        self.gatv23 = GATv2Conv(hidden_channels*heads, out_channels, concat=False,
                             heads=1, dropout=dropout)  
    
    def forward(self, x, edge_index):
        x = self.gatv21(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.7, training=self.training)
        x = self.gatv22(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.7, training=self.training)
        x = self.gatv23(x, edge_index)
        return x

In [33]:
def GATv2_model():
    torch.manual_seed(123)
    return GATv2(in_channels=num_features, hidden_channels=32, 
               out_channels=num_classes, heads=16, dropout=0.7).to(device)

In [34]:
tuner = HyperparameterTuner(GATv2_model, optimizer_lr=0.0004, epoches=345, scheduler_factor=0.3, 
                            scheduler_patience=1000, output_interval=15, lambda_reg=2)
tuner.run()

Epoch: 000, Con Loss: 9.30, Sup Loss: 2.10, Train Acc: 0.22, Val Loss: 1.92, Val Acc: 0.27
Epoch: 015, Con Loss: 9.01, Sup Loss: 2.04, Train Acc: 0.34, Val Loss: 1.86, Val Acc: 0.34
Epoch: 030, Con Loss: 8.72, Sup Loss: 2.05, Train Acc: 0.40, Val Loss: 1.80, Val Acc: 0.37
Epoch: 045, Con Loss: 8.67, Sup Loss: 1.85, Train Acc: 0.55, Val Loss: 1.62, Val Acc: 0.51
Epoch: 060, Con Loss: 8.65, Sup Loss: 1.64, Train Acc: 0.67, Val Loss: 1.38, Val Acc: 0.64
Epoch: 075, Con Loss: 8.65, Sup Loss: 1.53, Train Acc: 0.81, Val Loss: 1.19, Val Acc: 0.75
Epoch: 090, Con Loss: 8.60, Sup Loss: 1.35, Train Acc: 0.85, Val Loss: 1.05, Val Acc: 0.82
Epoch: 105, Con Loss: 8.59, Sup Loss: 1.26, Train Acc: 0.86, Val Loss: 0.93, Val Acc: 0.82
Epoch: 120, Con Loss: 8.57, Sup Loss: 1.19, Train Acc: 0.87, Val Loss: 0.84, Val Acc: 0.80
Epoch: 135, Con Loss: 8.55, Sup Loss: 1.07, Train Acc: 0.90, Val Loss: 0.76, Val Acc: 0.82
Epoch: 150, Con Loss: 8.56, Sup Loss: 1.08, Train Acc: 0.90, Val Loss: 0.70, Val Acc: 0.81

In [35]:
evaluator = ModelEvaluator(GATv2_model, optimizer_lr=0.0004, epoches=345, scheduler_factor=0.3, 
                            scheduler_patience=1000, lambda_reg=2)
evaluator.evaluate()

Fold 1 - Val Accuracy: 0.800, Val Loss: 0.519
Fold 2 - Val Accuracy: 0.778, Val Loss: 0.695
Fold 3 - Val Accuracy: 0.889, Val Loss: 0.363
Fold 4 - Val Accuracy: 0.879, Val Loss: 0.502
Fold 5 - Val Accuracy: 0.838, Val Loss: 0.504
Average Val Accuracy: 0.8368, Average Val Loss: 0.5168


### Transformer

In [36]:
from torch_geometric.nn import TransformerConv

In [37]:
class Transformer(torch.nn.Module):
    def __init__(self, in_channels, out_channels, hidden_channels, heads, dropout):
        super().__init__()
        self.conv1 = TransformerConv(in_channels, hidden_channels, heads=heads, dropout=dropout)
        self.conv2 = TransformerConv(hidden_channels*heads, out_channels, heads=1, concat=False, dropout=dropout)
    
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.7, training=self.training)
        x = self.conv2(x, edge_index)
        return x

In [38]:
def Transformer_model():
    torch.manual_seed(123)
    return Transformer(in_channels=num_features, hidden_channels=64, 
               out_channels=num_classes, heads=8, dropout=0.7).to(device)

In [39]:
tuner = HyperparameterTuner(Transformer_model, optimizer_lr=0.0008, epoches=300, scheduler_factor=0.5, 
                            scheduler_patience=1000, output_interval=15, lambda_reg=3)
tuner.run()

Epoch: 000, Con Loss: 10.01, Sup Loss: 1.94, Train Acc: 0.30, Val Loss: 1.91, Val Acc: 0.27
Epoch: 015, Con Loss: 8.69, Sup Loss: 1.78, Train Acc: 0.29, Val Loss: 1.83, Val Acc: 0.29
Epoch: 030, Con Loss: 8.53, Sup Loss: 1.75, Train Acc: 0.31, Val Loss: 1.82, Val Acc: 0.29
Epoch: 045, Con Loss: 8.49, Sup Loss: 1.47, Train Acc: 0.71, Val Loss: 1.55, Val Acc: 0.56
Epoch: 060, Con Loss: 8.43, Sup Loss: 1.21, Train Acc: 0.77, Val Loss: 1.29, Val Acc: 0.69
Epoch: 075, Con Loss: 8.30, Sup Loss: 0.99, Train Acc: 0.87, Val Loss: 1.08, Val Acc: 0.79
Epoch: 090, Con Loss: 8.23, Sup Loss: 0.86, Train Acc: 0.87, Val Loss: 0.92, Val Acc: 0.80
Epoch: 105, Con Loss: 8.17, Sup Loss: 0.74, Train Acc: 0.88, Val Loss: 0.83, Val Acc: 0.80
Epoch: 120, Con Loss: 8.10, Sup Loss: 0.63, Train Acc: 0.88, Val Loss: 0.76, Val Acc: 0.80
Epoch: 135, Con Loss: 8.10, Sup Loss: 0.59, Train Acc: 0.88, Val Loss: 0.71, Val Acc: 0.81
Epoch: 150, Con Loss: 8.05, Sup Loss: 0.53, Train Acc: 0.88, Val Loss: 0.67, Val Acc: 0.8

In [40]:
evaluator = ModelEvaluator(Transformer_model, optimizer_lr=0.0008, epoches=300, scheduler_factor=0.5, 
                            scheduler_patience=1000, lambda_reg=3)
evaluator.evaluate()

Fold 1 - Val Accuracy: 0.830, Val Loss: 0.530
Fold 2 - Val Accuracy: 0.848, Val Loss: 0.585
Fold 3 - Val Accuracy: 0.859, Val Loss: 0.388
Fold 4 - Val Accuracy: 0.838, Val Loss: 0.465
Fold 5 - Val Accuracy: 0.848, Val Loss: 0.513
Average Val Accuracy: 0.8448, Average Val Loss: 0.4959


### APPNP

In [41]:
from torch_geometric.nn import APPNP
from torch.nn import Linear

In [42]:
class APPNP1(torch.nn.Module):
    def __init__(self, in_channels, out_channels, hidden_channels, K, alpha, dropout):
        super().__init__()
        self.lin1 = Linear(in_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, out_channels)
        self.appnp = APPNP(K=K, alpha=alpha, dropout=dropout)
    
    def forward(self, x, edge_index):
        x = self.lin1(x)
        x = F.relu(x)
        x = F.dropout(x, p=0.7, training=self.training)
        x = self.lin2(x)
        x = self.appnp(x, edge_index)
        return x

In [43]:
def APPNP_model():
    torch.manual_seed(123)
    return APPNP1(in_channels=num_features, hidden_channels=128, 
               out_channels=num_classes, K=2, alpha=0.1, dropout=0.7).to(device)

In [52]:
tuner = HyperparameterTuner(APPNP_model, optimizer_lr=0.005, epoches=360, scheduler_factor=0.4, 
                            scheduler_patience=1000, output_interval=15, lambda_reg=3)
tuner.run()

Epoch: 000, Con Loss: 9.57, Sup Loss: 1.95, Train Acc: 0.11, Val Loss: 1.94, Val Acc: 0.12
Epoch: 015, Con Loss: 8.51, Sup Loss: 1.96, Train Acc: 0.29, Val Loss: 1.86, Val Acc: 0.30
Epoch: 030, Con Loss: 8.50, Sup Loss: 1.83, Train Acc: 0.45, Val Loss: 1.82, Val Acc: 0.43
Epoch: 045, Con Loss: 8.49, Sup Loss: 1.76, Train Acc: 0.60, Val Loss: 1.71, Val Acc: 0.51
Epoch: 060, Con Loss: 8.48, Sup Loss: 1.60, Train Acc: 0.84, Val Loss: 1.53, Val Acc: 0.78
Epoch: 075, Con Loss: 8.45, Sup Loss: 1.49, Train Acc: 0.90, Val Loss: 1.33, Val Acc: 0.81
Epoch: 090, Con Loss: 8.46, Sup Loss: 1.54, Train Acc: 0.90, Val Loss: 1.16, Val Acc: 0.82
Epoch: 105, Con Loss: 8.43, Sup Loss: 1.39, Train Acc: 0.92, Val Loss: 1.04, Val Acc: 0.86
Epoch: 120, Con Loss: 8.43, Sup Loss: 1.45, Train Acc: 0.93, Val Loss: 0.93, Val Acc: 0.85
Epoch: 135, Con Loss: 8.42, Sup Loss: 1.42, Train Acc: 0.94, Val Loss: 0.85, Val Acc: 0.84
Epoch: 150, Con Loss: 8.42, Sup Loss: 1.30, Train Acc: 0.95, Val Loss: 0.77, Val Acc: 0.85

In [51]:
evaluator = ModelEvaluator(APPNP_model, optimizer_lr=0.005, epoches=360, scheduler_factor=0.4, 
                            scheduler_patience=1000, lambda_reg=3)
evaluator.evaluate()

Fold 1 - Val Accuracy: 0.800, Val Loss: 0.518
Fold 2 - Val Accuracy: 0.818, Val Loss: 0.689
Fold 3 - Val Accuracy: 0.859, Val Loss: 0.398
Fold 4 - Val Accuracy: 0.838, Val Loss: 0.482
Fold 5 - Val Accuracy: 0.838, Val Loss: 0.513
Average Val Accuracy: 0.8307, Average Val Loss: 0.5199


### ChebConv

In [46]:
from torch_geometric.nn import ChebConv

In [47]:
class ChebConvModel(torch.nn.Module):
    def __init__(self, in_channels, out_channels, hidden_channels, K):
        super(ChebConvModel, self).__init__()
        self.conv1 = ChebConv(in_channels, hidden_channels, K)
        self.conv2 = ChebConv(hidden_channels, out_channels, K)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.7, training=self.training)
        x = self.conv2(x, edge_index)
        return x

In [48]:
def ChebConv_model():
    torch.manual_seed(123)
    return ChebConvModel(in_channels=num_features, hidden_channels=256,
               out_channels=num_classes, K=2).to(device)

In [49]:
tuner = HyperparameterTuner(ChebConv_model, optimizer_lr=0.0005, epoches=230, scheduler_factor=0.5,
                            scheduler_patience=1000, output_interval=13, lambda_reg=1.5)
tuner.run()

Epoch: 000, Con Loss: 9.90, Sup Loss: 2.05, Train Acc: 0.21, Val Loss: 1.95, Val Acc: 0.21
Epoch: 013, Con Loss: 9.69, Sup Loss: 1.50, Train Acc: 0.57, Val Loss: 1.56, Val Acc: 0.43
Epoch: 026, Con Loss: 9.22, Sup Loss: 1.18, Train Acc: 0.64, Val Loss: 1.38, Val Acc: 0.52
Epoch: 039, Con Loss: 9.01, Sup Loss: 0.92, Train Acc: 0.81, Val Loss: 1.16, Val Acc: 0.65
Epoch: 052, Con Loss: 8.80, Sup Loss: 0.71, Train Acc: 0.88, Val Loss: 0.94, Val Acc: 0.77
Epoch: 065, Con Loss: 8.69, Sup Loss: 0.51, Train Acc: 0.89, Val Loss: 0.79, Val Acc: 0.83
Epoch: 078, Con Loss: 8.55, Sup Loss: 0.42, Train Acc: 0.91, Val Loss: 0.69, Val Acc: 0.83
Epoch: 091, Con Loss: 8.47, Sup Loss: 0.34, Train Acc: 0.94, Val Loss: 0.63, Val Acc: 0.83
Epoch: 104, Con Loss: 8.43, Sup Loss: 0.28, Train Acc: 0.95, Val Loss: 0.58, Val Acc: 0.82
Epoch: 117, Con Loss: 8.30, Sup Loss: 0.23, Train Acc: 0.97, Val Loss: 0.55, Val Acc: 0.83
Epoch: 130, Con Loss: 8.28, Sup Loss: 0.19, Train Acc: 0.97, Val Loss: 0.52, Val Acc: 0.85

In [50]:
evaluator = ModelEvaluator(ChebConv_model, optimizer_lr=0.0005, epoches=230, scheduler_factor=0.5,
                            scheduler_patience=1000, lambda_reg=1.5)
evaluator.evaluate()

Fold 1 - Val Accuracy: 0.810, Val Loss: 0.604
Fold 2 - Val Accuracy: 0.838, Val Loss: 0.535
Fold 3 - Val Accuracy: 0.899, Val Loss: 0.400
Fold 4 - Val Accuracy: 0.848, Val Loss: 0.552
Fold 5 - Val Accuracy: 0.828, Val Loss: 0.564
Average Val Accuracy: 0.8448, Average Val Loss: 0.5310
